# 2.1 Getting your data + feature engineering

In [367]:
import pandas as pd
from datetime import datetime
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

df=pd.read_csv("bank_transactions.csv")

In [368]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048567 entries, 0 to 1048566
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   TransactionID            1048567 non-null  object 
 1   CustomerID               1048567 non-null  object 
 2   CustomerDOB              1045170 non-null  object 
 3   CustGender               1047467 non-null  object 
 4   CustLocation             1048416 non-null  object 
 5   CustAccountBalance       1046198 non-null  float64
 6   TransactionDate          1048567 non-null  object 
 7   TransactionTime          1048567 non-null  int64  
 8   TransactionAmount (INR)  1048567 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 72.0+ MB


## Na analysis

In [369]:
df.shape

(1048567, 9)

In [370]:
df.isna().sum()

TransactionID                 0
CustomerID                    0
CustomerDOB                3397
CustGender                 1100
CustLocation                151
CustAccountBalance         2369
TransactionDate               0
TransactionTime               0
TransactionAmount (INR)       0
dtype: int64

Since we have to create new variables starting with the ones that we have, we think that rows with NA values
are useless.
Also we can notice that rows which contain NA values are in a small number compared to the shape of the dataframe.
According to these  we delete them.

In [371]:
df.dropna(inplace=True)
df.shape

(1041614, 9)

## Change the data type

In [372]:
df[["CustomerDOB","TransactionDate","TransactionTime"]].head(1)

,CustomerDOB,TransactionDate,TransactionTime
0,10/1/94,2/8/16,143207


Since there are variables with date and times not in the proper format we will change the type of those variables.

In [373]:
df.CustomerDOB=pd.to_datetime(df.CustomerDOB)
df.TransactionDate=pd.to_datetime(df.TransactionDate)

In [374]:
df[["CustomerDOB","TransactionDate"]].head(1)

,CustomerDOB,TransactionDate
0,1994-10-01,2016-02-08


In [375]:
df.TransactionTime = df.TransactionTime.apply(lambda x: datetime.strptime(str(x).zfill(6), '%H%M%S').time())

Now the time variables seems to be correct.

In [376]:
df[["CustomerDOB","TransactionDate","TransactionTime"]].head(1)

,CustomerDOB,TransactionDate,TransactionTime
0,1994-10-01,2016-02-08,14:32:07


## Delete costumer that results born in 1800

In [377]:
df.drop(df[df.CustomerDOB.dt.year == 1800].index, axis=0, inplace=True)
df.shape

(985322, 9)

There was 56292 transactions belonging to customer born in 1800, or registered by error born in 1800.

# Create new variables


## Grouping by customer ID
Each row will be a different client.
There are some issue with gender's variable so we are escluding it for now.

In [378]:
customer=df.groupby("CustomerID").sum()
customer=customer.rename(columns={"TransactionAmount (INR)":"TOT TransactionAmount (INR)"})
customer.drop(columns=["CustAccountBalance"],inplace=True)

We have deleted because is a features that must not be summed.

### Create  "Number of transactions"

In [379]:
number_transaction=pd.DataFrame(df.groupby("CustomerID").count().TransactionID)
number_transaction=number_transaction.rename(columns={"TransactionID":"Number_of_transaction"})
customer=pd.merge(customer,number_transaction,on="CustomerID")

### Create  "Number of transactions" bigger than 100 USD

In [380]:
number_transaction=pd.DataFrame(df[df["TransactionAmount (INR)"]> 100].groupby("CustomerID").count().TransactionID)
number_transaction=number_transaction.rename(columns={"TransactionID":"Number_of_transaction_100"})
customer=pd.merge(customer,number_transaction,on="CustomerID", how='left')

#Put 0 instead of NaN
customer.Number_of_transaction_100=customer.Number_of_transaction_100.fillna(0)

### Average of the transactions

In [381]:
means=pd.DataFrame(df.groupby("CustomerID")["TransactionAmount (INR)"].mean())
means=means.rename(columns={"TransactionAmount (INR)":"Average_transaction_amount"})
customer=pd.merge(customer,means,on="CustomerID")

### Average balance

In [382]:
average=pd.DataFrame(df.groupby("CustomerID")["CustAccountBalance"].mean())
average=average.rename(columns={"CustAccountBalance":"Average_amount_balance"})
customer=pd.merge(customer,average,on="CustomerID")

### Utilisation 
Average difference between the balance and the transaction amount for each customer (this is mainly known in the banking world as utilisation).

In [383]:
df['Utilisation'] = df['CustAccountBalance'] - df['TransactionAmount (INR)']

average=pd.DataFrame(df.groupby("CustomerID")['Utilisation'].mean())
average=average.rename(columns={"Utilisation":"Average_Utilisation"})
customer=pd.merge(customer,average,on="CustomerID")

### Gender of the customer

In [384]:
sesso=df[["CustomerID","CustGender"]].drop_duplicates(subset=["CustomerID"])
customer=pd.merge(customer,sesso,on="CustomerID")

### Most frequent location of the customer

In [385]:
#We create a subset of the dataset.
data=df[['CustomerID','CustLocation']]

#Create a new variable
data['Most_common_location'] = data.groupby('CustomerID')['CustLocation'].transform(lambda x: x.mode()[0])

#We drop the duplicates in order to avoid error during the merging. That's the reason of the creation of the subset.
data.drop_duplicates(subset=['CustomerID'],inplace=True)

#Merging data
customer=pd.merge(customer,data[["CustomerID",'Most_common_location']],on="CustomerID")

customer.shape

(839081, 9)

## Create other 20 features

### CustomerAge

In [386]:
#We create a subset of the dataset.
data=df[["CustomerID","CustomerDOB"]]

data.loc[data.CustomerDOB.dt.year > 2000, 'CustomerDOB'] = data.loc[data.CustomerDOB.dt.year > 2000, 'CustomerDOB'] - pd.DateOffset(years = 100)

#We drop the duplicates in order to avoid error during the merging. That's the reason of the creation of the subset.
data.drop_duplicates(subset=['CustomerID'],inplace=True)

#Create a new variable
data['CustomerAge'] = (( pd.to_datetime('today') - data.CustomerDOB ) / np.timedelta64(1, 'Y')).round(0)

#Change the type of the column
data['CustomerAge'] = data['CustomerAge'].astype(int)

#Merging
customer=pd.merge(customer,data[["CustomerID",'CustomerAge']],on="CustomerID")

customer.shape

(839081, 10)

### Binary for major costumer and Age_in_2016
The feature "Major" will be 1 is the costumer was major in 2016, 0 if the costumer was under 18 years old in 2016.


In [387]:
#Create a new variable
data['Age_in_2016'] = (( pd.to_datetime('2016-12-31') - data.CustomerDOB ) / np.timedelta64(1, 'Y')).round(0)

data['Major']=1
data.loc[data["Age_in_2016"] <18,'Major']=0

#Change the type of the column
data["Age_in_2016"] = data["Age_in_2016"].astype(int)

#Merging
customer=pd.merge(customer,data[["CustomerID","Age_in_2016",'Major']],on="CustomerID")

customer.shape

(839081, 12)

In [388]:
customer.to_pickle("/Users/mattia/Desktop/ADM(Aris)/HM4/customer.pkl")

### Old customer
The feature "OLD" will be 1 is the costumer in 2016 was 70 years old or more.

In [389]:
#Create a new variable
data['Old']=0
data.loc[data["Age_in_2016"] >= 70,'Old']=1

#Merging
customer=pd.merge(customer,data[["CustomerID",'Old']],on="CustomerID")

customer.shape

(839081, 13)

### 

In [392]:
df

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),Utilisation
0,T1,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,14:32:07,25.0,17794.05
1,T2,C2142763,2057-04-04,M,JHAJJAR,2270.69,2016-02-08,14:18:58,27999.0,-25728.31
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,14:27:12,459.0,17415.44
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,14:27:14,2060.0,864443.21
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-02-08,18:11:56,1762.5,4951.93
...,...,...,...,...,...,...,...,...,...,...
1048562,T1048563,C8020229,1990-08-04,M,NEW DELHI,7635.19,2016-09-18,18:48:24,799.0,6836.19
1048563,T1048564,C6459278,1992-02-20,M,NASHIK,27311.42,2016-09-18,18:37:34,460.0,26851.42
1048564,T1048565,C6412354,1989-05-18,M,HYDERABAD,221757.06,2016-09-18,18:33:13,770.0,220987.06
1048565,T1048566,C6420483,1978-08-30,M,VISAKHAPATNAM,10117.87,2016-09-18,18:47:06,1000.0,9117.87


# Export the data

In [390]:
customer.to_pickle("/Users/mattia/Desktop/ADM(Aris)/HM4/customer.pkl")